In [7]:
# Import the importatnt libraries
import os
import numpy as np
from torchvision import models
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Dataset
import matplotlib.pyplot as plt
from torchvision import transforms
import pandas as pd
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import f1_score
import copy

In [8]:
# ignoring the warning messages
import warnings
from IPython.display import display
warnings.filterwarnings('ignore')

In [9]:
torch.manual_seed(42)
# batch size for all the future processes
batch_size = 32

In [10]:
data = np.load("/content/pneumoniamnist.npz")

In [11]:
train_images, train_labels = data["train_images"], data["train_labels"]

In [12]:
val_images, val_labels = data["val_images"], data["val_labels"]

In [13]:
test_images, test_labels = data["test_images"], data["test_labels"]

In [14]:
train_images.shape

(3882, 28, 28)

In [15]:
train_labels.shape

(3882, 1)

In [16]:
val_images.shape

(524, 28, 28)

In [17]:
val_labels.shape

(524, 1)

In [18]:
test_images.shape

(624, 28, 28)

In [19]:
test_labels.shape

(624, 1)

In [20]:
class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Convert image to torch tensor
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)  # if using BCEWithLogitsLoss

        # Ensure image has shape [1, H, W]
        if image.ndim == 2:
            image = image.unsqueeze(0)
        elif image.ndim == 3 and image.shape[-1] in [1, 3]:
            image = image.permute(2, 0, 1)

        # Convert 1-channel grayscale to 3-channel RGB by repeating
        if image.shape[0] == 1:
            image = image.repeat(3, 1, 1)

        if self.transform:
            image = self.transform(image)

        return image, label


In [21]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),         # Resize to match InceptionV3 input size
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Normalize to [-1, 1] range (optional)
])

In [22]:
train_dataset = ImageDataset(train_images, train_labels, transform=transform)
val_dataset = ImageDataset(val_images, val_labels, transform=transform)
test_dataset = ImageDataset(test_images, test_labels, transform=transform)

In [23]:
# Data loader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
# Validation loader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [24]:
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, num_workers=4)

In [25]:
def train(model, train_loader, val_loader, criterion, optimizer, device, epoch, num_epochs, scheduler, threshold=0.5):
    model.train()
    running_loss = 0.0
    num_correct_preds = 0.0
    total_preds = 0.0
    train_preds = []
    train_labels = []

    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as tepoch:
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device).float().squeeze(1)  # ✅ Force shape: [B]

            optimizer.zero_grad()

            # InceptionV3 returns (main_output, aux_output) during training
            main_output, aux_output = model(images)
            main_output = main_output.squeeze(1)
            aux_output = aux_output.squeeze(1)

            # Compute losses
            loss_main = criterion(main_output, labels)
            loss_aux = criterion(aux_output, labels)
            loss = loss_main + 0.4 * loss_aux  # Combine main and auxiliary loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Predictions from main output
            probs = torch.sigmoid(main_output)  # shape: [batch_size]
            preds = (probs > threshold).long()

            train_preds.extend(preds.detach().cpu().numpy())
            train_labels.extend(labels.detach().cpu().numpy())

            num_correct_preds += (preds == labels.long()).sum().item()
            total_preds += labels.size(0)

            tepoch.set_postfix(
                loss=running_loss / (tepoch.n + 1),
                accuracy=f"{(num_correct_preds / len(train_loader.dataset)) * 100:.2f}%"
            )
            tepoch.update(1)

    scheduler.step()

    f1 = f1_score(train_labels, train_preds, average='weighted')
    train_accuracy = (num_correct_preds / total_preds) * 100

    return train_accuracy, running_loss / len(train_loader), f1


In [26]:
def validate(model, val_loader, criterion, device, threshold=0.5):
    model.eval()
    running_loss = 0.0
    num_correct_preds = 0.0
    total_preds = 0.0
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float()  # ✅ Force shape: [B]

            outputs = model(images)  # shape: [batch_size]
            loss = criterion(outputs, labels)

            running_loss += loss.item()

            probs = torch.sigmoid(outputs)
            preds = (probs > threshold).long()

            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

            num_correct_preds += (preds == labels.long()).sum().item()
            total_preds += labels.size(0)

    f1 = f1_score(val_labels, val_preds, average='weighted')
    val_accuracy = (num_correct_preds / total_preds) * 100

    return val_accuracy, running_loss / len(val_loader), f1


In [27]:
import torch
import copy

def training_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, scheduler, model_name, threshold=0.5):
    best_accuracy = 0.0
    best_model = None

    for epoch in range(num_epochs):
        # train the model
        train_accuracy, train_loss, train_f1 = train(
            model, train_loader, val_loader, criterion, optimizer, device, epoch, num_epochs, scheduler, threshold=threshold
        )

        # validate the model
        val_accuracy, val_loss, val_f1 = validate(
            model, val_loader, criterion, device, threshold=threshold
        )

        # Save the best model based on validation accuracy
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, f'best_model_{model_name}_epoch{epoch+1}.pth')

            print(f'✅ Best model saved at epoch {epoch+1}/{num_epochs} with:')
            print(f'   🟢 Train Acc: {train_accuracy:.2f}%, Loss: {train_loss:.4f}, F1: {train_f1:.4f}')
            print(f'   🔵 Val   Acc: {val_accuracy:.2f}%, Loss: {val_loss:.4f}, F1: {val_f1:.4f}')

        torch.cuda.empty_cache()

    return model, best_model


In [28]:
# using the pre-trained model with importing the pretrained weight efficente_v2_m
# from torchvision.models import Inception_V3_Weights
model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 152MB/s]


In [29]:
# InceptionV3
model.fc = nn.Linear(model.fc.in_features, 1)
model.AuxLogits.fc = nn.Linear(model.AuxLogits.fc.in_features, 1)

In [30]:
# for param in model.parameters():
#     param.requires_grad = False  # Freeze all layers

# # Unfreeze classifier layers
# for param in model.fc.parameters():
#     param.requires_grad = True

# for param in model.AuxLogits.fc.parameters():
#     param.requires_grad = True


In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = nn.DataParallel(model, device_ids=[0, 1])  # Use GPUs 0 and 1
model = model.to(device)

In [32]:
# num_epochs = 15
num_epochs = 5
criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.NAdam(model.parameters(), lr=1e-4)
# use the optimizer wiht SGD, momentum  and nestrov momentum
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, nesterov=True)
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# scheduler for the learning rate
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.1e-6)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

last_epoch_model, best_model = training_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, scheduler, 'inception_v3', threshold=0.5)


Epoch 1/5: 100%|██████████| 122/122 [00:56<00:00,  2.17batch/s, accuracy=94.56%, loss=0.243]


✅ Best model saved at epoch 1/5 with:
   🟢 Train Acc: 94.56%, Loss: 0.2429, F1: 0.9469
   🔵 Val   Acc: 91.60%, Loss: 0.1889, F1: 0.9108


Epoch 2/5: 100%|██████████| 122/122 [00:56<00:00,  2.17batch/s, accuracy=98.82%, loss=0.0664]


✅ Best model saved at epoch 2/5 with:
   🟢 Train Acc: 98.82%, Loss: 0.0664, F1: 0.9881
   🔵 Val   Acc: 96.95%, Loss: 0.1019, F1: 0.9690


Epoch 5/5: 100%|██████████| 122/122 [00:55<00:00,  2.18batch/s, accuracy=99.97%, loss=0.00892]


✅ Best model saved at epoch 5/5 with:
   🟢 Train Acc: 99.97%, Loss: 0.0089, F1: 0.9997
   🔵 Val   Acc: 97.14%, Loss: 0.0805, F1: 0.9710


In [37]:
validate(best_model, test_loader, criterion, device, threshold=0.5)

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [40]:
# Load the best model state dictionary into a new model instance
model.load_state_dict(best_model)

test_accuracy, test_loss, test_f1 = validate(model, test_loader, criterion, device, threshold=0.5)

print(f'🚀 Test Accuracy: {test_accuracy:.2f}%, Loss: {test_loss:.4f}, F1: {test_f1:.4f}')

🚀 Test Accuracy: 81.73%, Loss: 0.7133, F1: 0.7995
